<a href="https://colab.research.google.com/github/choycoy/Final/blob/main/prediction_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd
q_df = pd.read_csv("/content/drive/My Drive/final/questions.csv",sep=",")
q_df['question_id'] = q_df['question_id'].str.replace('q', '')
q_df

In [ ]:
df1 = pd.read_csv("/content/drive/My Drive/final/combined_with_filename__1.csv",sep=",")
df2 = pd.read_csv("/content/drive/My Drive/final/combined_with_filename__2.csv",sep=",")
df3 = pd.read_csv("/content/drive/My Drive/final/combined_with_filename__3.csv",sep=",")
df4 = pd.read_csv("/content/drive/My Drive/final/combined_with_filename__4.csv",sep=",")
df5 = pd.read_csv("/content/drive/My Drive/final/combined_with_filename__5.csv",sep=",")
df6 = pd.read_csv("/content/drive/My Drive/final/combined_with_filename__6.csv",sep=",")
df7 = pd.read_csv("/content/drive/My Drive/final/combined_with_filename__7.csv",sep=",")
df8 = pd.read_csv("/content/drive/My Drive/final/combined_with_filename__8.csv",sep=",")
df9 = pd.read_csv("/content/drive/My Drive/final/combined_with_filename__9.csv",sep=",")
df10 = pd.read_csv("/content/drive/My Drive/final/combined_with_filename__10.csv",sep=",")
df11 = pd.read_csv("/content/drive/My Drive/final/combined_with_filename__11.csv",sep=",")
df12 = pd.read_csv("/content/drive/My Drive/final/combined_with_filename__12.csv",sep=",")
df13 = pd.read_csv("/content/drive/My Drive/final/combined_with_filename__13.csv",sep=",")
df14 = pd.read_csv("/content/drive/My Drive/final/combined_with_filename__14.csv",sep=",")
df15 = pd.read_csv("/content/drive/My Drive/final/combined_with_filename__15.csv",sep=",")
df16 = pd.read_csv("/content/drive/My Drive/final/combined_with_filename__16.csv",sep=",")
df17 = pd.read_csv("/content/drive/My Drive/final/combined_with_filename__17.csv",sep=",")
df18 = pd.read_csv("/content/drive/My Drive/final/combined_with_filename__18.csv",sep=",")
df19 = pd.read_csv("/content/drive/My Drive/final/combined_with_filename__19.csv",sep=",")
df20 = pd.read_csv("/content/drive/My Drive/final/combined_with_filename__20.csv",sep=",")
df21 = pd.read_csv("/content/drive/My Drive/final/combined_with_filename__21.csv",sep=",")
df22 = pd.read_csv("/content/drive/My Drive/final/combined_with_filename__22.csv",sep=",")
df23 = pd.read_csv("/content/drive/My Drive/final/combined_with_filename__23.csv",sep=",")
df24 = pd.read_csv("/content/drive/My Drive/final/combined_with_filename__24.csv",sep=",")
df25 = pd.read_csv("/content/drive/My Drive/final/combined_with_filename__25.csv",sep=",")
df26 = pd.read_csv("/content/drive/My Drive/final/combined_with_filename__26.csv",sep=",")
df27 = pd.read_csv("/content/drive/My Drive/final/combined_with_filename__27.csv",sep=",")
df28 = pd.read_csv("/content/drive/My Drive/final/combined_with_filename__28.csv",sep=",")


df = pd.concat([df1, df2, df3, df4, df5, df6, df7, df8,df9, df10, df11, df12, df13, df14, df15, df16, df17, df18, df19, df20,df21, df22,df23, df24, df25, df26, df27, df28])

In [ ]:
df['question_id'] = df['question_id'].astype('int64')
q_df['question_id'] = q_df['question_id'].astype('int64')

# Merge the dataframes on 'question_id'
merged_df = pd.merge(df, q_df[['question_id', 'correct_answer']], on='question_id', how='left')

# Create the 'is_correct' column based on the comparison between 'user_answer' and 'correct_answer'
merged_df['is_correct'] = (merged_df['user_answer'] == merged_df['correct_answer']).astype(int)

# Now merged_df will have a new column 'is_correct' indicating if the user_answer is correct or not
print(merged_df)

In [ ]:
# Sort the dataframe by timestamp to ensure the data is in chronological order
merged_df.sort_values(by=['user_id', 'timestamp'], inplace=True)

# Calculate Correctness Rate and Interaction Count for each user
merged_df['correct_count'] = merged_df.groupby('user_id')['is_correct'].cumsum()  # Cumulative sum of correct answers for each user
print(merged_df['correct_count'])

merged_df['interaction_count'] = merged_df.groupby('user_id').cumcount() + 1  # Cumulative count of interactions for each user
print(merged_df['interaction_count'])

merged_df['correctness_rate'] = merged_df['correct_count'] / merged_df['interaction_count']  # Calculate the correctness rate for each interaction
merged_df['average_elapsed_time'] = merged_df.groupby('user_id')['elapsed_time'].transform('mean')

# Drop unnecessary columns if needed
merged_df.drop(['correct_count', 'interaction_count'], axis=1, inplace=True)

In [ ]:
merged_df['next_is_correct'] = merged_df['is_correct'].shift(-1)
merged_df.dropna(subset=['next_is_correct'], inplace=True)

In [ ]:
# adjust features = ['average_elapsed_time', 'correctness_rate'] or features = ['correctness_rate']
features = ['average_elapsed_time', 'correctness_rate']

In [ ]:
from sklearn.model_selection import train_test_split

X = merged_df[features]
y = merged_df['next_is_correct']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
model = DecisionTreeClassifier(random_state=42)
model.fit(X_train, y_train)

# Make predictions on the testing data
y_pred = model.predict(X_test)

# Evaluate the model's performance
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print("Accuracy: {:.2f}".format(accuracy))
print("Precision: {:.2f}".format(precision))
print("Recall: {:.2f}".format(recall))
print("F1-score: {:.2f}".format(f1))

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
model = KNeighborsClassifier(n_neighbors=5)
model.fit(X_train, y_train)

# Make predictions on the testing data
y_pred = model.predict(X_test)

# Evaluate the model's performance
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1)

In [ ]:
from sklearn.ensemble import BaggingClassifier
bag = BaggingClassifier(base_estimator=model, n_estimators=50, random_state=42)

# Train the model on the training data
bag.fit(X_train, y_train)

# Make predictions on the testing data
y_pred = bag.predict(X_test)

# Evaluate the model's performance
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print("Accuracy: {:.2f}".format(accuracy))
print("Precision: {:.2f}".format(precision))
print("Recall: {:.2f}".format(recall))
print("F1-score: {:.2f}".format(f1))

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
classifier_boost = AdaBoostClassifier(base_estimator=model, n_estimators=50, random_state=42)

# Train the model on the training data
classifier_boost.fit(X_train, y_train)

# Make predictions on the testing data
y_pred = classifier_boost.predict(X_test)

# Evaluate the model's performance
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print("Accuracy: {:.2f}".format(accuracy))
print("Precision: {:.2f}".format(precision))
print("Recall: {:.2f}".format(recall))
print("F1-score: {:.2f}".format(f1))

In [ ]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(n_estimators=100, random_state=42)

# Fit the model to the training data
model.fit(X_train, y_train)

# Make predictions on the testing data
y_pred = model.predict(X_test)

# Evaluate the model's performance
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print("Accuracy: {:.2f}".format(accuracy))
print("Precision: {:.2f}".format(precision))
print("Recall: {:.2f}".format(recall))
print("F1-score: {:.2f}".format(f1))
